Краткий аутлайн решения:
- На сдвинутых спутниках делаем shift на 1 и засовываем в ответ 

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import seaborn as sns
import gc
from tqdm import tqdm

from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split

from warnings import filterwarnings
filterwarnings('ignore')

%matplotlib inline

PATH = '../'

In [5]:
%%time
train = pd.read_csv(PATH + 'train.csv')
test = pd.read_csv(PATH + 'Track 1/test.csv')
submission = pd.read_csv(PATH + 'Track 1/submission.csv')

Wall time: 2.56 s


In [6]:
test_unchanged = test.copy()
train_unchanged = train.copy()

### LB Score Calc

Для подсчета скора просто используйте функцию getLBScore

In [7]:
def smape(satellite_predicted_values, satellite_true_values): 
    # the division, addition and subtraction are pointwise 
    return np.mean(np.abs(satellite_predicted_values - satellite_true_values) / (np.abs(satellite_predicted_values) + np.abs(satellite_true_values)))

In [8]:
def getLBScore(real, predicted):
    smp = []
    for sat_id in real['sat_id'].unique():
        rv = real[real['sat_id'] == sat_id][['x', 'y', 'z', 'Vx', 'Vy', 'Vz']].values
        sym = predicted[predicted['sat_id'] == sat_id][['x', 'y', 'z', 'Vx', 'Vy', 'Vz']].values
        smp.append(smape(sym.ravel(), rv.ravel()))
    meanSmape = np.mean(smp)
    lbScore = 100 * (1 - meanSmape)
    print('Mean SMAPE ', meanSmape, '; LB score ', lbScore)

### Solution

In [22]:
shiftedSat = [1,
 2,
 3,
 4,
 25,
 27,
 32,
 38,
 41,
 45,
 59,
 63,
 84,
 86,
 92,
 98,
 100,
 101,
 109,
 112,
 118,
 123,
 124,
 125,
 126,
 133,
 135,
 149,
 151,
 155,
 173,
 177,
 186,
 188,
 195,
 208,
 213,
 223,
 230,
 236,
 241,
 242,
 243,
 247,
 248,
 251,
 264,
 270,
 278,
 348,
 363,
 375,
 380,
 382,
 385,
 390,
 392,
 407,
 409,
 415,
 416,
 418,
 422,
 437,
 440,
 450,
 468,
 476,
 477,
 480,
 483,
 495,
 502,
 505,
 508,
 536,
 539,
 542,
 549,
 558,
 565,
 575,
 580,
 595,
 596,
 597]

In [ ]:
# LB ~ 67.8 дельта 1.6 На валидации дельта: 0.9
# Скор при использовании симулирующих значений на трейне как предсказаний
trainSym = train.copy()
trainSym.iloc[:, 3:9] = trainSym.iloc[:, 9:].values

for sat_id in trainSym['sat_id'].unique():
    if sat_id in shiftedSat:
        satData = trainSym[trainSym['sat_id'] == sat_id]
        satDataShifted = satData.shift(1).fillna(0)
        satData.iloc[int(0.3 * satData.shape[0]):, 9:] = satDataShifted.iloc[int(0.3 * satData.shape[0]):, 9:].values # 0.5
        trainSym.iloc[satData.index, :] = satData.values
        trainSym.iloc[:, 3:9] = trainSym.iloc[:, 9:].values
        getLBScore(train, trainSym)

Mean SMAPE  0.15088303250684615 ; LB score  84.91169674931538
Mean SMAPE  0.15066798803470566 ; LB score  84.93320119652942
Mean SMAPE  0.1505170588181162 ; LB score  84.94829411818839
Mean SMAPE  0.1503432047095872 ; LB score  84.96567952904128
Mean SMAPE  0.15011723877302638 ; LB score  84.98827612269736
Mean SMAPE  0.15012527763652192 ; LB score  84.9874722363478
Mean SMAPE  0.15000919773555396 ; LB score  84.9990802264446
Mean SMAPE  0.1497874842094348 ; LB score  85.02125157905652
Mean SMAPE  0.14964662527557657 ; LB score  85.03533747244234
Mean SMAPE  0.14947129834755904 ; LB score  85.0528701652441
Mean SMAPE  0.14931852060090245 ; LB score  85.06814793990975
Mean SMAPE  0.14914572436381857 ; LB score  85.08542756361814
Mean SMAPE  0.14891300682329137 ; LB score  85.10869931767087
Mean SMAPE  0.1487482757854197 ; LB score  85.12517242145803
Mean SMAPE  0.14861581010425962 ; LB score  85.13841898957403
Mean SMAPE  0.14847199672351447 ; LB score  85.15280032764856
Mean SMAPE  0.1

In [31]:
testSym = test.copy()
for sat_id in tqdm(test['sat_id'].unique()):
    if sat_id in shiftedSat:
        satData = test[test['sat_id'] == sat_id]
        satDataShifted = satData.shift(1).fillna(0)
        satData.iloc[int(0.5 * satData.shape[0]):, 3:] = satDataShifted.iloc[int(0.5 * satData.shape[0]):, 3:].values
        testSym.iloc[satData.index, :] = satData.values



  0%|                                                                                                                                                                                                               | 0/300 [00:00<?, ?it/s]

  0%|▋                                                                                                                                                                                                      | 1/300 [00:00<01:25,  3.48it/s]

  1%|█▎                                                                                                                                                                                                     | 2/300 [00:00<01:23,  3.55it/s]

  1%|█▉                                                                                                                                                                                                     | 3/300 [00:00<01:22,  3.61it/s]

  1%|██▋                                      

 37%|████████████████████████████████████████████████████████████████████████▉                                                                                                                            | 111/300 [00:10<00:17, 10.62it/s]

 38%|███████████████████████████████████████████████████████████████████████████▌                                                                                                                         | 115/300 [00:10<00:16, 11.36it/s]

 40%|██████████████████████████████████████████████████████████████████████████████▏                                                                                                                      | 119/300 [00:10<00:15, 11.85it/s]

 41%|█████████████████████████████████████████████████████████████████████████████████▍                                                                                                                   | 124/300 [00:11<00:13, 13.06it/s]

 42%|███████████████████████████████████████████

 84%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                               | 252/300 [00:21<00:05,  9.07it/s]

 85%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                             | 255/300 [00:21<00:04,  9.41it/s]

 86%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                            | 257/300 [00:22<00:05,  8.44it/s]

 86%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                           | 259/300 [00:22<00:05,  7.93it/s]

 91%|███████████████████████████████████████████

In [36]:
submission.iloc[:, 1:] = testSym.iloc[:, 3:].values

In [38]:
submission.to_csv('submission.csv', index = None)